# 构建您的第一个电子商务推荐器

本笔记本将指导您完成构建域数据集组和推荐器的步骤，该推荐器将根据为虚构的零售商店数据集生成的数据返回产品推荐。目标是根据特定用户推荐相关产品。

此综合数据来自 [Retail Demo Store 项目](https://github.com/aws-samples/retail-demo-store)。点击链接，了解有关数据和潜在用途的更多信息。

# 如何使用笔记本

此代码被分解为如下所示的单元格。在本页顶部有一个三角形 Run（运行）按钮，您可以单击该按钮以执行每个单元格，并移至下一个单元格；也可以按 `Shift` + `Enter` 组合键，在单元格中执行并移至下一个单元格。

当一个单元格执行时，您会注意到该单元格运行时旁边有一行显示 `*`，或者它会更新为一个数字，以指示在执行完单元格中的所有代码后完成执行的最后一个单元格。

只需按照下面的说明执行单元格，即可通过案例优化推荐器开始使用 Amazon Personalize。

## 导入
Python 附带有许多库，我们需要导入这些库以及已安装的库来帮助我们，比如 [boto3](https://aws.amazon.com/sdk-for-python/)（适用于 Python 的 AWS SDK）和 [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)，它们都是核心数据科学工具。

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

接下来，您需要验证您的环境是否可以成功地与 Amazon Personalize 进行通信，下面的行就可以实现这一目的。

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 指定 S3 存储桶和数据输出位置

Amazon Personalize 需要 S3 存储桶来充当数据源。下面的代码将创建具有唯一 `bucket_name` 的存储桶。

Amazon S3 存储桶必须与 Amazon Personalize 资源位于同一区域。 

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
    
except Exception as e:
    print (e)
    print("Bucket already exists. Using bucket", bucket_name)

## 下载、准备和上载训练数据

首先，我们需要下载数据（训练数据）。在本教程中，我们将使用零售商店数据集中的购买历史记录。此数据集包含用户 ID、项目 ID、客户和项目之间的交互，以及此交互发生的时间（时间戳） 

### 下载并浏览数据集

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/items.csv .
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/interactions.csv .

数据集已成功下载为 Electronics_Store_purchase_history.csv

让我们通过查看数据集的特性来了解有关数据集的更多信息

In [ ]:
df = pd.read_csv('./interactions.csv')
df

In [ ]:
df.EVENT_TYPE.value_counts()

In [ ]:
def convert_event_type(event_type_in_some_format):
    if(event_type_in_some_format == "ProductViewed"):
        return "View"
    if(event_type_in_some_format == "OrderCompleted"):
        return "Purchase"
    else:
        return event_type_in_some_format

df['EVENT_TYPE'] = df['EVENT_TYPE'].apply(convert_event_type)

In [ ]:
df.EVENT_TYPE.value_counts()

电子商务推荐器要求您提供特定的 EVENT_TYPE 值，以了解交互背景，由此我们将修改交互 EVENTYPE 列

In [ ]:
df.info()

从上面的 2 个单元格中，我们了解到我们的数据有 9 列，675004 行，标题为：ITEM_ID、USER_ID、EVENT_TYPE、TIMESTAMP 和 DISCOUNT。

要与 Amazon Personalize 交互模式兼容，此数据集的列标题需要与 Amazon Personalize 默认列名称兼容（请参阅[此处](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)的列名称）

## 准备交互数据


### 删除列

此数据集中的某些列不会为我们的模型添加值，因此需要从此数据集中删除列。列，如 *discount*。

In [ ]:
test=df.drop(columns=['DISCOUNT'])
df=test
df.sample(10)

在下面的单元格中，我们将把清理后的数据写入名为 "final_training_data.csv 的文件中

In [ ]:
df.to_csv("cleaned_training_data.csv")

### 上载到 S3
现在我们的训练数据已准备好用于 Amazon Personalize，下一步是将数据上载到之前创建的 S3 存储桶

In [ ]:
interactions_file_path = 'cleaned_training_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path


## 配置 S3 存储桶和 IAM 角色

到目前为止，我们已下载、操作数据并将数据保存到运行此 Jupyter 笔记本的实例所连接的 Amazon EBS 实例中。但是，Amazon Personalize 需要一个 S3 存储桶充当您的数据源以及用于访问该存储桶的 IAM 角色。让我们把一切都安排好。


## 设置 S3 存储桶策略
Amazon Personalize 需要能够读取 S3 存储桶的内容。因此，添加可实现此操作的存储桶策略。

注意：确保您用于在此笔记本中运行代码的角色具有修改 S3 存储桶策略所需的权限。

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## 创建并等待数据集组
Personalize 中最大的分组是数据集组，这将隔离您的数据、事件跟踪器、解决方案、推荐器和活动。把共享通用数据集合的内容分组在一起。您可以随意更改下面的名称。

### 创建数据集组

In [ ]:
response = personalize.create_dataset_group(
    name='personalize_ecomemerce_ds_group',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

等待数据集组状态变为 ACTIVE
在我们可以在以下任何项目中使用数据集组之前，数据集组必须处于活跃状态，请执行下面的单元格并等待项目显示为活跃状态。

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 创建交互模式
Personalize 如何理解您的数据的核心组件来自下面定义的模式。此配置指导服务如何消化通过 CSV 文件提供的数据。注意列和类型与您在上面创建的文件中的内容对齐。

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
            
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## 创建数据集
在组之后，接下来要创建的是实际数据集。

### 创建交互数据集

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## 创建 Personalize 角色
此外，Amazon Personalize 需要能够担任 AWS 中的角色，以便拥有执行某些任务的权限，下面的行授予了此权限。

注意：确保您用于在此笔记本中运行代码的角色具有创建角色所需的权限。

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## 导入数据
您之前创建了数据集组和数据集来存储您的信息，现在您将执行一个导入作业，从 S3 将数据加载到 Amazon Personalize 中，用于构建您的模型。
### 创建交互数据集导入作业

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

等待数据集导入作业状态变为 ACTIVE
导入作业可能需要一会儿才能完成，请等待直至看到它在下面处于活跃状态。

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 选择推荐器使用案例

每个域都有不同的使用案例。您需要为特定使用案例创建推荐器，并且每个使用案例获得推荐的要求都不同。


In [ ]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain. 
display (available_recipes['recipes'])

我们将创建类型为"查看 X 的客户也查看了"的推荐器。此推荐器为客户根据您指定的项目而同时查看的项目提供推荐。在此使用案例中，Amazon Personalize 会根据您指定的用户 ID 和 `Purchase` 事件自动筛选用户已购买的项目。

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'viewed_x_also_viewed_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
  datasetGroupArn = dataset_group_arn
)
viewed_x_also_viewed_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

我们将创建第二个类型为"推荐给您"的推荐器。此类型的推荐器根据您指定的用户为项目提供个性化推荐。在此使用案例中，Amazon Personalize 会根据您指定的用户 ID 和 `Purchase` 事件自动筛选用户已购买的项目。

[每个域的更多使用案例](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

我们等待直至推荐器完成创建且状态变为 `ACTIVE`。我们定期检查推荐器的状态

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours

while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = viewed_x_also_viewed_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(viewed_x_also_viewed_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(viewed_x_also_viewed_arn))
        

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Customers who viewed X also viewed" Recommender build is still in progress')
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommended_for_you_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommended_for_you_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommended_for_you_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Recommended for you" Recommender build is still in progress')
        
    time.sleep(60)

## 通过推荐器获得推荐
现在推荐器已经接受过训练，我们来看看我们可以为用户提供的推荐！

In [ ]:
# reading the original data in order to have a dataframe that has both item_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./items.csv')
items_df.sample(10)

In [ ]:
def get_item_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item description.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Error obtaining item description"

让我们使用"查看 X 的客户也查看了"推荐器获得一些推荐：

In [ ]:
# use a random valid id for a quick sanity check, modify the line of code bellow to a valid id in your dataset
get_item_by_id("c72257d4-430b-4eb7-9de3-28396e593381", items_df)

In [ ]:
# First pick a user
test_user_id = "777"

# Select a random item
test_item_id = "8fbe091c-f73c-4727-8fe7-d27eabd17bea" # a random item: 8fbe091c-f73c-4727-8fe7-d27eabd17bea

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = viewed_x_also_viewed_arn,
    itemId = test_item_id,
    userId = test_user_id,
    numResults = 10
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []

for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_item_by_id(test_item_id, items_df)])

pd.options.display.max_rows =10
display(user_recommendations_df)

从推荐器获得推荐，返回"推荐给您"：

In [ ]:
# First pick a user
test_user_id = "777" 

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)


user_recommendations_df = pd.DataFrame(recommendation_list, columns = [test_user_id])

pd.options.display.max_rows =20
display(user_recommendations_df)

## 审核
使用上述代码，您已成功训练了一个深度学习模型，可根据之前的用户行为生成项目推荐。您已为两个基本使用案例创建了两个推荐器。
接下来，您可以调整此代码以创建其他推荐器。

## 适用于下一个笔记本的注意事项：
您将需要一些值以用于下一个笔记本，请执行下面的单元格来存储这些值，以便在 `Clean_Up_Resources.ipynb` 笔记本中加以使用。

这将覆盖为这些变量存储的任何数据，并将数据设置为此笔记本中指定的值。 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

如果您已运行 `Building_Your_First_Recommender_Video_On_Demand.ipynb` 笔记本，请确保在 `Building_Your_First_Recommender_Video_On_Demand.ipynb` 笔记本中重新运行上一步骤，并重新运行 `Clean_Up_Resources.ipynb`，以删除在运行 `Clean_Up_Resources.ipynb` 后在该笔记本中创建的资源以及此处创建的资源。